In [11]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import sklearn.linear_model as skl_lm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

import plotly.express as px

%matplotlib inline
plt.style.use('seaborn-white')

In [4]:
PROJECT_ROOT_DIR = "/Users/daniel421/Desktop/STAT_724/ds_724/Final_Project"
# CHAPTER_ID = "Lin_regression"
# IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
# os.makedirs(IMAGES_PATH, exist_ok=True)

In [5]:
import os
#UP_DIR = '/Users/daniel421/Desktop/STAT_724/ISLR_data'
csv_file = os.path.join(PROJECT_ROOT_DIR,'listings_5.csv')
abnb = pd.read_csv(csv_file, low_memory=False)
abnb.head()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,accommodates,bedrooms,beds,...,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost2,host_identity_verified2,has_availability2,instant_bookable2,dmy_Hotel room,dmy_Private room,dmy_Shared room,Borough
0,35%,18%,8.0,8.0,Midtown,40.75356,-73.98559,1,NaN,1.0,...,0,0.33,0,1,1,0,0,0,0,Manhattan
1,21%,76%,1.0,1.0,Bedford-Stuyvesant,40.68494,-73.95765,3,1.0,3.0,...,0,4.91,0,1,1,0,0,0,0,Brooklyn
2,100%,100%,1.0,1.0,Bedford-Stuyvesant,40.68535,-73.95512,2,1.0,1.0,...,0,0.53,0,1,1,0,0,1,0,Brooklyn
3,100%,25%,1.0,1.0,Sunset Park,40.66265,-73.99454,4,2.0,2.0,...,0,0.02,0,1,1,0,0,0,0,Brooklyn
4,100%,100%,1.0,1.0,Midtown,40.76457,-73.98317,2,1.0,1.0,...,0,3.70,0,0,1,0,0,1,0,Manhattan


In [7]:
len(abnb)

15991

In [8]:
scaler.fit_transform(abnb)

NameError: name 'scaler' is not defined

In [10]:
import plotly.express as px

## EDA and Visualizations

Make a dataframe for Queens, allowing any and all prices of the listings

In [13]:
abnb_queens_df = abnb[abnb.Borough == 'Queens']
abnb_queens_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,accommodates,bedrooms,beds,...,calculated_host_listings_count_shared_rooms,reviews_per_month,host_is_superhost2,host_identity_verified2,has_availability2,instant_bookable2,dmy_Hotel room,dmy_Private room,dmy_Shared room,Borough
16,100%,94%,1.0,1.0,Long Island City,40.74757,-73.94571,4,1.0,2.0,...,0,3.15,1,0,1,0,0,1,0,Queens
18,90%,82%,5.0,5.0,Jamaica,40.67144,-73.76501,3,1.0,2.0,...,0,0.81,0,1,1,0,0,1,0,Queens
64,100%,33%,1.0,1.0,Sunnyside,40.74523,-73.92318,1,1.0,1.0,...,0,0.24,0,1,1,0,0,1,0,Queens
74,100%,36%,5.0,5.0,Ridgewood,40.70309,-73.89963,16,5.0,10.0,...,0,0.53,1,1,1,0,0,0,0,Queens
81,90%,82%,5.0,5.0,Jamaica,40.67144,-73.76501,2,1.0,1.0,...,0,0.39,0,1,1,0,0,1,0,Queens


## Begin by implementing K-fold cross validation